In [1]:
from utils.model import MNmodelPartAM_Dueling
import pickle
import torch
import os
from matplotlib import pyplot as plt
from utils.learner import Segmenter, RLLearner
from utils.env_utils import load_env
import cv2
from tqdm import tqdm
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_build = 'NAVIndoor/maze' #environment path
env, behavior_name,channel_env = load_env(train_build,10,0,0,0.3)

model = MNmodelPartAM_Dueling(scale=3,output=4,dropout=0,n_actions = 4,am = 20,aes = 20, ahes = 150,n_frames=3)
model.load_state_dict(torch.load('checkpoint/checkpoint.pt',map_location=device))


Learner = RLLearner(env=env,
                    model=model,
                    target_model=model,
                    action_mapper = {0:[1,0],1:[0,1],2:[0,-1],3:[-1,0]},
                    segmenter = Segmenter(True), #Segmenter argument is ground segmentation (with value 0.5)
                    channel_env = channel_env,
                    using_seg_input = True,
                    action_memory = 20
) 
Learner.init_state()


[UnityMemory] Configuration Parameters - Can be set up in boot.config
    "memorysetup-bucket-allocator-granularity=16"
    "memorysetup-bucket-allocator-bucket-count=8"
    "memorysetup-bucket-allocator-block-size=4194304"
    "memorysetup-bucket-allocator-block-count=1"
    "memorysetup-main-allocator-block-size=16777216"
    "memorysetup-thread-allocator-block-size=16777216"
    "memorysetup-gfx-main-allocator-block-size=16777216"
    "memorysetup-gfx-thread-allocator-block-size=16777216"
    "memorysetup-cache-allocator-block-size=4194304"
    "memorysetup-typetree-allocator-block-size=2097152"
    "memorysetup-profiler-bucket-allocator-granularity=16"
    "memorysetup-profiler-bucket-allocator-bucket-count=8"
    "memorysetup-profiler-bucket-allocator-block-size=4194304"
    "memorysetup-profiler-bucket-allocator-block-count=1"
    "memorysetup-profiler-allocator-block-size=16777216"
    "memorysetup-profiler-editor-allocator-block-size=1048576"
    "memorysetup-temp-allocator-siz

In [ ]:
video_path = 'video_episode.avi'

frames = Learner.clip(1000)
video = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'XVID'),15, (128,128)) #path, encoding, frames per second, image size
for image in tqdm(frames):
    video.write(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
video.release()